<a href="https://colab.research.google.com/github/mjmousavi97/MachinLearning-TehranUni-2024/blob/main/Projects/Final%20Project/src/extract_audio_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
import librosa
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
def extract_features(audio, sr):
    """
    Extracts a set of audio features from a given audio signal.

    Features extracted:
        - MFCCs (13 coefficients)
        - Spectral Centroid
        - Spectral Bandwidth
        - Spectral Contrast
        - Zero Crossing Rate
        - Root Mean Square (RMS) Energy

    Parameters:
        audio (np.ndarray): 1D array containing the audio signal.
        sr (int): Sample rate of the audio signal.

    Returns:
        np.ndarray: 2D array where each row corresponds to a feature and
                    columns correspond to time frames.
    """

    # Extract 13 MFCC coefficients
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)

    # Compute spectral centroid (center of mass of the spectrum)
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)

    # Compute spectral bandwidth (spread of the spectrum)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)

    # Compute spectral contrast (difference between peaks and valleys in spectrum)
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)

    # Compute zero crossing rate (rate at which signal crosses zero)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)

    # Compute root mean square (RMS) energy
    energy = librosa.feature.rms(y=audio)

    # Stack all features vertically into a single 2D array
    return np.vstack([mfccs, spectral_centroid, spectral_bandwidth, spectral_contrast, zero_crossing_rate, energy])


In [17]:
import numpy as np

def process_features_mean(features):
    """
    Compute the mean of each feature over time.

    Parameters:
        features (np.ndarray): 2D array where rows are features and columns are time frames.

    Returns:
        np.ndarray: 1D array containing the mean value of each feature.
    """
    return np.mean(features, axis=1)


def process_features_max(features):
    """
    Compute the maximum value of each feature over time.

    Parameters:
        features (np.ndarray): 2D array where rows are features and columns are time frames.

    Returns:
        np.ndarray: 1D array containing the maximum value of each feature.
    """
    return np.max(features, axis=1)


def process_features_std(features):
    """
    Compute the standard deviation of each feature over time.

    Parameters:
        features (np.ndarray): 2D array where rows are features and columns are time frames.

    Returns:
        np.ndarray: 1D array containing the standard deviation of each feature.
    """
    return np.std(features, axis=1)


In [18]:
import numpy as np

def extract_and_process_features(audio, sr):
    """
    Extract audio features from a given signal and compute summary statistics.

    Steps:
        1. Extract time-series features (MFCCs, spectral features, zero crossing rate, energy).
        2. Compute mean, maximum, and standard deviation for each feature over time.
        3. Concatenate all summary statistics into a single feature vector.

    Parameters:
        audio (np.ndarray): 1D array containing the audio signal.
        sr (int): Sample rate of the audio signal.

    Returns:
        np.ndarray: 1D array containing concatenated mean, max, and std of all features.
    """

    # Step 1: Extract raw time-series features
    raw_features = extract_features(audio, sr)

    # Step 2: Compute summary statistics for each feature
    mean_features = process_features_mean(raw_features)
    max_features = process_features_max(raw_features)
    std_features = process_features_std(raw_features)

    # Step 3: Combine all summary statistics into a single feature vector
    combined_features = np.hstack((mean_features, max_features, std_features))

    return combined_features


In [20]:
audio_folder = '/content/drive/MyDrive/filtered_audio'
file_paths = [os.path.join(audio_folder, name) for name in os.listdir(audio_folder) if name.endswith('.mp3')]

In [27]:
data = []

# Loop through each audio file
for file_path in file_paths:

    # Get the file name
    file_name = os.path.basename(file_path)

    # Load audio file with its original sample rate
    audio, sr = librosa.load(file_path, sr=None)

    # Extract and process features (mean, max, std)
    features = extract_and_process_features(audio, sr)

    # Combine file name and features into a single row
    row = [file_name] + features.tolist()

    # Append the row to the data list
    data.append(row)

In [33]:
# Create a DataFrame from the collected data
num_features = len(features)  # Number of extracted features
columns = ["file_name"] + [f"feature_{i+1}" for i in range(num_features)]
df = pd.DataFrame(data, columns=columns)

# Display DataFrame shape and first 5 rows
print("DataFrame shape:", df.shape)
print(df.head())

DataFrame shape: (362, 73)
                      file_name   feature_1  ...  feature_71  feature_72
0     HW1_Q5_159403005_male.mp3 -366.249108  ...    0.049621    0.024161
1     HW1_Q6_810103146_male.mp3 -588.872355  ...    0.053655    0.001649
2     HW1_Q3_810100107_male.mp3 -248.265962  ...    0.081746    0.040632
3     HW1_Q6_810103112_male.mp3 -459.328542  ...    0.083750    0.011283
4  HW1_intro_810103269_male.mp3 -523.087209  ...    0.110765    0.003758

[5 rows x 73 columns]


In [34]:
# Extract 'id' and 'gender' from the file_name
df['id'] = df['file_name'].apply(lambda x: x.split('_')[2])       # 3rd part is student id
df['gender'] = df['file_name'].apply(lambda x: x.split('_')[3].split('.')[0])  # 4th part before '.mp3' is gender

print(df[['file_name', 'id', 'gender']].head())


                      file_name         id gender
0     HW1_Q5_159403005_male.mp3  159403005   male
1     HW1_Q6_810103146_male.mp3  810103146   male
2     HW1_Q3_810100107_male.mp3  810100107   male
3     HW1_Q6_810103112_male.mp3  810103112   male
4  HW1_intro_810103269_male.mp3  810103269   male


In [35]:
df

,file_name,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,id,gender
0,HW1_Q5_159403005_male.mp3,-366.249108,123.833206,-39.281316,-3.645372,-14.236462,-24.647222,-31.240593,-28.083876,-11.787792,-19.155800,-21.617325,-12.576558,-11.550668,1098.578935,931.717311,31.458133,17.745527,19.052858,17.270252,22.606164,34.589556,31.418423,0.092792,0.028269,-62.339756,222.304321,57.358253,92.531570,56.289066,44.627670,19.316505,22.750093,32.089390,30.901318,19.803516,23.726931,24.093174,3723.666807,2149.680885,69.681067,43.718379,52.326429,37.745177,50.863605,62.351572,58.642350,0.461426,0.176666,89.280238,45.271203,32.184560,23.337839,15.428002,15.538591,13.862964,15.719300,11.708783,11.851424,9.257057,7.952642,8.202302,407.325966,264.491658,4.984766,6.091566,5.542606,3.975120,4.768754,10.272667,10.214406,0.049621,0.024161,159403005,male
1,HW1_Q6_810103146_male.mp3,-588.872355,125.100890,-24.485900,17.280049,-17.532478,-15.952730,-8.525033,-24.814333,-17.578835,-7.722893,-14.690779,-9.007000,-9.898188,1127.280127,1004.813593,29.140668,15.567080,18.387043,16.768360,17.967235,40.403627,26.589896,0.091130,0.002013,-369.997925,226.658569,59.781487,108.319427,50.332573,41.953995,45.069851,27.946768,32.512421,41.251137,20.219318,25.872999,21.223755,4323.986613,2390.845667,66.443435,40.278967,49.883871,38.797492,58.505613,63.156190,62.214248,0.580566,0.012147,80.169959,41.740907,29.709328,26.747445,18.248844,16.189226,13.946703,16.765053,13.101916,11.089955,9.432461,7.665231,8.001237,493.620498,327.165166,5.399740,5.262488,5.084067,3.782120,4.485039,12.977027,12.317271,0.053655,0.001649,810103146,male
2,HW1_Q3_810100107_male.mp3,-248.265962,121.320401,-46.533523,38.722161,-20.318026,-4.829107,-20.965555,-20.801089,-12.501102,-16.596553,-9.682689,-11.885527,-12.375692,1399.226836,1162.607626,30.662920,15.068322,18.221885,16.140488,18.133022,30.236092,38.827641,0.128954,0.057374,-72.560013,213.673981,35.920990,126.229431,36.285103,50.260117,33.793427,36.622116,31.129602,28.667927,24.331516,24.062309,19.925785,4089.055678,2255.293619,59.799836,37.205021,40.507646,32.101330,35.231713,58.516324,60.931445,0.516113,0.229701,81.879078,45.652086,26.430169,28.896845,19.723219,16.526925,18.584140,18.099094,13.268143,13.943461,10.550746,9.797526,11.669768,609.529568,265.112855,6.179784,5.363383,5.501842,3.568206,3.210981,8.379653,6.303463,0.081746,0.040632,810100107,male
3,HW1_Q6_810103112_male.mp3,-459.328542,91.579614,-10.711723,13.661453,-14.005515,-10.544291,-18.587549,-22.645896,-17.796876,-7.201046,-7.647645,-14.424867,-7.901971,1162.255461,967.379866,30.382727,15.516150,19.316222,18.310051,19.346356,36.303765,25.191217,0.111324,0.011061,-200.495193,234.586029,75.630257,139.978058,51.604824,61.072449,39.446671,27.658894,26.259041,34.148842,32.931053,27.877110,23.267609,4489.047410,2327.945227,69.805278,50.582943,56.178059,56.664800,60.172430,62.803227,58.058127,0.623047,0.082323,112.679608,64.451395,24.723825,27.839658,19.388542,16.846025,16.555195,17.073263,14.192902,10.232751,9.974516,11.378676,8.492335,713.132327,394.184828,5.912024,5.244909,6.550245,6.715201,6.954696,11.738149,11.001539,0.083750,0.011283,810103112,male
4,HW1_intro_810103269_male.mp3,-523.087209,92.231330,-38.374928,-13.531664,-50.827071,2.403356,-41.845082

In [36]:
# Define the path and name of the CSV file to save
output_csv = "/content/drive/MyDrive/filtered_audio/processed_features.csv"

# Save the DataFrame as a CSV file
df.to_csv(output_csv, index=False)  # index=False prevents saving the row indices

# Print confirmation message
print(f"DataFrame saved as CSV at: {output_csv}")

DataFrame saved as CSV at: /content/drive/MyDrive/filtered_audio/processed_features.csv
